# Generating Song Recommendations Using Spotify

### Connecting to the API

In [19]:
# Import all libraries used throughout the notebook
#import os
# TRY TO CREATE ENV FOR API KEYS from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [26]:
# Construct variables to hold access credentials needed in order to connect 
# with the SpotifY API

my_id = "57619a02d735467d97e9f5fcc6b264cc"
my_secret_key = "f4e9db72af4e46868b418ba4e68beb7e"

credentials_manager = SpotifyClientCredentials(client_id = my_id, client_secret = my_secret_key)
sp = spotipy.Spotify(auth_manager = credentials_manager)

### Extracting my Music

In [ ]:
get a list of track ids fronm my playlist, using the ids use the audio_features function to extra music features information and graph the features

In [ ]:
# Get song ids for the songs in my playlist.



In [ ]:
https://www.youtube.com/watch?v=oNyaiWgqKDI

In [ ]:
https://developer.spotify.com/documentation/web-api/tutorials/code-flow

In [ ]:
https://developer.spotify.com/documentation/web-api/tutorials/getting-started

In [ ]:
https://levelup.gitconnected.com/music-analysis-with-spotify-api-59c080734c6e

In [ ]:
https://developer.spotify.com/documentation/web-api/howtos/web-app-profile

In [ ]:
https://developer.spotify.com/documentation/web-api/reference/get-recommendations

In [ ]:
https://developer.spotify.com/documentation/web-api/concepts/scopes#playlist-read-private

In [ ]:
https://developer.spotify.com/documentation/web-api/tutorials/code-flow